In [1]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

#Here's your endpoint after you sign up
ENDPOINT = "https://your-project.cognitiveservices.azure.com/"

# Replace with a valid key of your own.
training_key = ""
prediction_key = ""
prediction_resource_id = ""

publish_iteration_name = "detectModel"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
project = trainer.create_project("Your Project name", domain_id=obj_detection_domain.id)
print("done")

Creating project...


In [22]:
#All your tags here
wheat = trainer.create_tag(project.id, "wheat")
wheat.id

'f09da180-0c15-4600-b8d0-8d2a64c37497'

In [215]:
import pandas as pd
import numpy as np
import os
import math

In [185]:
IMG_DIR ='train/'
TRAIN = "train_labels.csv"
TEST_DIR = 'test/'
train_df = pd.read_csv(TRAIN)
train_df.info(),train_df.shape
train_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147343 entries, 0 to 147342
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  147343 non-null  int64  
 1   width       147343 non-null  int64  
 2   height      147343 non-null  int64  
 3   source      147343 non-null  object 
 4   filename    147343 non-null  object 
 5   class       147343 non-null  object 
 6   xmin        147343 non-null  float64
 7   ymin        147343 non-null  float64
 8   xmax        147343 non-null  float64
 9   ymax        147343 non-null  float64
 10  w           147343 non-null  float64
 11  h           147343 non-null  float64
dtypes: float64(6), int64(3), object(3)
memory usage: 13.5+ MB


,Unnamed: 0,width,height,source,filename,class,xmin,ymin,xmax,ymax,w,h
0,0,1024,1024,usask_1,b6ab77fd7.jpg,wheat,834.0,222.0,890.0,258.0,56.0,36.0
1,1,1024,1024,usask_1,b6ab77fd7.jpg,wheat,226.0,548.0,356.0,606.0,130.0,58.0
2,2,1024,1024,usask_1,b6ab77fd7.jpg,wheat,377.0,504.0,451.0,664.0,74.0,160.0
3,3,1024,1024,usask_1,b6ab77fd7.jpg,wheat,834.0,95.0,943.0,202.0,109.0,107.0
4,4,1024,1024,usask_1,b6ab77fd7.jpg,wheat,26.0,144.0,150.0,261.0,124.0,117.0


In [198]:
#NORMALIZE COORDINATES
train_df['xmin'] = (train_df['xmin']/train_df['width'])
train_df['xmax'] = (train_df['xmax']/train_df['width'])
train_df['w'] = (train_df['w']/train_df['width'])
train_df['ymin'] = (train_df['ymin']/train_df['height'])
train_df['ymax'] = (train_df['ymax']/train_df['height'])
train_df['h'] = (train_df['h']/train_df['height'])
train_df.head(3)

,Unnamed: 0,width,height,source,filename,class,xmin,ymin,xmax,ymax,w,h
0,0,1024,1024,usask_1,b6ab77fd7.jpg,wheat,0.814453,0.216797,0.869141,0.251953,0.054688,0.035156
1,1,1024,1024,usask_1,b6ab77fd7.jpg,wheat,0.220703,0.535156,0.347656,0.591797,0.126953,0.056641
2,2,1024,1024,usask_1,b6ab77fd7.jpg,wheat,0.368164,0.492188,0.440430,0.648438,0.072266,0.156250


In [199]:
#df_aux have all the records for given file_name
def addBoxes(df_aux,file_name,tagged_images_with_regions):
    regions = []
    for index, row in df_aux.iterrows():
        x,y,w,h = row['xmin'],row['ymin'],row['w'],row['h']
        regions.append(Region(tag_id=wheat.id, left=x,top=y,width=w,height=h)) #We only have wheaat

    image_name = os.path.splitext(os.path.basename(file_name))[0]
    with open(IMG_DIR + file_name, mode="rb") as image_contents:
        tagged_images_with_regions.append(ImageFileCreateEntry(name=image_name, contents=image_contents.read(), regions=regions))
    return tagged_images_with_regions

In [148]:
#Upload images with boxes to cloud.
def uploadResults(tagged_images_with_regions):
    upload_result = trainer.create_images_from_files(project.id, images=tagged_images_with_regions)
    if not upload_result.is_batch_successful:
        print("Image batch upload failed.")
        for image in upload_result.images:
            print("Image status: ", image.status)
        exit(-1)
    else:
        print("Upload successfully")

In [ ]:
#batch size is important. When this was done, max batch size in Azure for free account was 64.
#keys are images file names
def uploadFromKeys(keys,df,batch_size):
    tagged_images_with_regions = []
    iteration = 1
    counter = 0
    print("Uploading. Will iterate {} times".format(str(math.ceil(len(keys)/batch_size))))
    for index in np.arange(len(keys)):
        if(counter==batch_size):
            print('It {}. Batch size reached... uploading results before proceeding'.format(str(iteration)))
            uploadResults(tagged_images_with_regions)
            tagged_images_with_regions = []
            counter=0
            iteration+=1

        filename = keys[index]
        group = df[df['filename'] == filename] #We get all boxes for that image
        tagged_images_with_regions = addBoxes(group,filename,tagged_images_with_regions)
        counter+=1
        
    print('Last Iteration... uploading results before proceeding')
    uploadResults(tagged_images_with_regions)
    tagged_images_with_regions = []
    print('Done')

In [239]:
def train(publish): #You won't be able to publish twice without unpublish from website.
    import time

    print ("Training...")
    iteration = trainer.train_project(project.id)
    while (iteration.status != "Completed"):
        iteration = trainer.get_iteration(project.id, iteration.id)
    #     print ("Training status: " + iteration.status)
        time.sleep(60)

    # The iteration is now trained. Publish it to the project endpoint
    if(publish):
        trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
    print ("Done!")

In [222]:
def runFromLowToHigh(low,high,batch_size):
    keys = list(np.unique(train_df['filename']))
    iteration_keys = keys[low:high]
    uploadFromKeys(iteration_keys,train_df,batch_size)

In [240]:
#We generate 4 trained iterations to compare results. There's no actual need to.
batch_size = 10
low = 0
high = 50
runFromLowToHigh(low,high,batch_size)
train(False)
low = high
high = 500
runFromLowToHigh(low,high,batch_size)
train(False)
low = high
high = 1500
runFromLowToHigh(low,high,batch_size)
train(False)
low = high
high = 3362
runFromLowToHigh(low,high,batch_size)
train(True)

Will iterate 5 times
It 1. Batch size reached... uploading results before proceeding
Upload successfully
It 2. Batch size reached... uploading results before proceeding
Upload successfully
It 3. Batch size reached... uploading results before proceeding
Upload successfully
It 4. Batch size reached... uploading results before proceeding
Upload successfully
Last Iteration... uploading results before proceeding
Upload successfully
Done
Training...
Done!
Will iterate 45 times
It 1. Batch size reached... uploading results before proceeding
Upload successfully
It 2. Batch size reached... uploading results before proceeding
Upload successfully
It 3. Batch size reached... uploading results before proceeding
Upload successfully
It 4. Batch size reached... uploading results before proceeding
Upload successfully
It 5. Batch size reached... uploading results before proceeding
Upload successfully
It 6. Batch size reached... uploading results before proceeding
Upload successfully
It 7. Batch size rea

Upload successfully
It 47. Batch size reached... uploading results before proceeding
Upload successfully
It 48. Batch size reached... uploading results before proceeding
Upload successfully
It 49. Batch size reached... uploading results before proceeding
Upload successfully
It 50. Batch size reached... uploading results before proceeding
Upload successfully
It 51. Batch size reached... uploading results before proceeding
Upload successfully
It 52. Batch size reached... uploading results before proceeding
Upload successfully
It 53. Batch size reached... uploading results before proceeding
Upload successfully
It 54. Batch size reached... uploading results before proceeding
Upload successfully
It 55. Batch size reached... uploading results before proceeding
Upload successfully
It 56. Batch size reached... uploading results before proceeding
Upload successfully
It 57. Batch size reached... uploading results before proceeding
Upload successfully
It 58. Batch size reached... uploading result

Upload successfully
It 44. Batch size reached... uploading results before proceeding
Upload successfully
It 45. Batch size reached... uploading results before proceeding
Upload successfully
It 46. Batch size reached... uploading results before proceeding
Upload successfully
It 47. Batch size reached... uploading results before proceeding
Upload successfully
It 48. Batch size reached... uploading results before proceeding
Upload successfully
It 49. Batch size reached... uploading results before proceeding
Upload successfully
It 50. Batch size reached... uploading results before proceeding
Upload successfully
It 51. Batch size reached... uploading results before proceeding
Upload successfully
It 52. Batch size reached... uploading results before proceeding
Upload successfully
It 53. Batch size reached... uploading results before proceeding
Upload successfully
It 54. Batch size reached... uploading results before proceeding
Upload successfully
It 55. Batch size reached... uploading result

Upload successfully
It 140. Batch size reached... uploading results before proceeding
Upload successfully
It 141. Batch size reached... uploading results before proceeding
Upload successfully
It 142. Batch size reached... uploading results before proceeding
Upload successfully
It 143. Batch size reached... uploading results before proceeding
Upload successfully
It 144. Batch size reached... uploading results before proceeding
Upload successfully
It 145. Batch size reached... uploading results before proceeding
Upload successfully
It 146. Batch size reached... uploading results before proceeding
Upload successfully
It 147. Batch size reached... uploading results before proceeding
Upload successfully
It 148. Batch size reached... uploading results before proceeding
Upload successfully
It 149. Batch size reached... uploading results before proceeding
Upload successfully
It 150. Batch size reached... uploading results before proceeding
Upload successfully
It 151. Batch size reached... uplo

PREDICT YOUR OWN IMAGES.

In [263]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

TRESHOLD = 0.5
predictions = {}
# Now there is a trained endpoint that can be used to make a prediction
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

for file in os.listdir(TEST_DIR):
    # Open the sample image and get back the prediction results.
    
    with open(os.path.join(TEST_DIR, file), mode="rb") as test_data:
        results = predictor.detect_image(project.id, publish_iteration_name, test_data)

    # Display the results.  
    print("PREDICTION FOR {}".format(file))
    predictions[file] = []
    for prediction in results.predictions:
        if (prediction.probability > TRESHOLD):
            predictions[file].append(prediction.probability)
            print("\t" + prediction.tag_name + ": {0:.2f}% bbox.left = {1:.2f}, bbox.top = {2:.2f}, bbox.width = {3:.2f}, bbox.height = {4:.2f}".format(prediction.probability * 100, prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height))
    print('---------------------------------------------------------------------------------------------')

PREDICTION FOR 2fd875eaa.jpg
	wheat: 94.59% bbox.left = 0.01, bbox.top = 0.00, bbox.width = 0.09, bbox.height = 0.05
	wheat: 67.84% bbox.left = 0.06, bbox.top = 0.00, bbox.width = 0.09, bbox.height = 0.03
	wheat: 95.72% bbox.left = 0.12, bbox.top = 0.00, bbox.width = 0.10, bbox.height = 0.06
	wheat: 98.78% bbox.left = 0.22, bbox.top = 0.00, bbox.width = 0.09, bbox.height = 0.05
	wheat: 55.32% bbox.left = 0.26, bbox.top = 0.00, bbox.width = 0.08, bbox.height = 0.02
	wheat: 66.51% bbox.left = 0.98, bbox.top = 0.00, bbox.width = 0.02, bbox.height = 0.03
	wheat: 91.66% bbox.left = 0.12, bbox.top = 0.04, bbox.width = 0.10, bbox.height = 0.06
	wheat: 97.78% bbox.left = 0.47, bbox.top = 0.02, bbox.width = 0.06, bbox.height = 0.05
	wheat: 98.05% bbox.left = 0.42, bbox.top = 0.07, bbox.width = 0.09, bbox.height = 0.06
	wheat: 75.58% bbox.left = 0.48, bbox.top = 0.07, bbox.width = 0.06, bbox.height = 0.05
	wheat: 57.09% bbox.left = 0.24, bbox.top = 0.15, bbox.width = 0.05, bbox.height = 0.05
	wh

PREDICTION FOR 51b3e36ab.jpg
	wheat: 88.94% bbox.left = 0.00, bbox.top = 0.00, bbox.width = 0.10, bbox.height = 0.04
	wheat: 63.34% bbox.left = 0.00, bbox.top = 0.01, bbox.width = 0.04, bbox.height = 0.10
	wheat: 71.97% bbox.left = 0.00, bbox.top = 0.06, bbox.width = 0.03, bbox.height = 0.08
	wheat: 88.66% bbox.left = 0.50, bbox.top = 0.17, bbox.width = 0.09, bbox.height = 0.06
	wheat: 86.78% bbox.left = 0.49, bbox.top = 0.19, bbox.width = 0.09, bbox.height = 0.06
	wheat: 94.81% bbox.left = 0.89, bbox.top = 0.19, bbox.width = 0.09, bbox.height = 0.06
	wheat: 70.99% bbox.left = 0.01, bbox.top = 0.27, bbox.width = 0.08, bbox.height = 0.09
	wheat: 96.84% bbox.left = 0.00, bbox.top = 0.35, bbox.width = 0.05, bbox.height = 0.07
	wheat: 50.87% bbox.left = 0.89, bbox.top = 0.36, bbox.width = 0.11, bbox.height = 0.06
	wheat: 83.29% bbox.left = 0.97, bbox.top = 0.34, bbox.width = 0.03, bbox.height = 0.08
	wheat: 54.62% bbox.left = 0.00, bbox.top = 0.37, bbox.width = 0.04, bbox.height = 0.08
	wh

PREDICTION FOR 53f253011.jpg
	wheat: 61.92% bbox.left = 0.98, bbox.top = 0.00, bbox.width = 0.02, bbox.height = 0.03
	wheat: 58.03% bbox.left = 0.02, bbox.top = 0.06, bbox.width = 0.09, bbox.height = 0.07
	wheat: 98.52% bbox.left = 0.14, bbox.top = 0.10, bbox.width = 0.08, bbox.height = 0.07
	wheat: 63.33% bbox.left = 0.47, bbox.top = 0.16, bbox.width = 0.07, bbox.height = 0.06
	wheat: 53.73% bbox.left = 0.61, bbox.top = 0.15, bbox.width = 0.08, bbox.height = 0.06
	wheat: 67.93% bbox.left = 0.61, bbox.top = 0.18, bbox.width = 0.08, bbox.height = 0.06
	wheat: 78.70% bbox.left = 0.47, bbox.top = 0.21, bbox.width = 0.08, bbox.height = 0.07
	wheat: 51.86% bbox.left = 0.91, bbox.top = 0.22, bbox.width = 0.06, bbox.height = 0.09
	wheat: 67.62% bbox.left = 0.96, bbox.top = 0.22, bbox.width = 0.04, bbox.height = 0.09
	wheat: 70.41% bbox.left = 0.16, bbox.top = 0.31, bbox.width = 0.10, bbox.height = 0.06
	wheat: 72.00% bbox.left = 0.34, bbox.top = 0.30, bbox.width = 0.09, bbox.height = 0.07
	wh

PREDICTION FOR aac893a91.jpg
	wheat: 55.03% bbox.left = 0.00, bbox.top = 0.00, bbox.width = 0.03, bbox.height = 0.03
	wheat: 93.70% bbox.left = 0.30, bbox.top = 0.00, bbox.width = 0.07, bbox.height = 0.06
	wheat: 59.27% bbox.left = 0.66, bbox.top = 0.00, bbox.width = 0.08, bbox.height = 0.02
	wheat: 69.97% bbox.left = 0.82, bbox.top = 0.00, bbox.width = 0.08, bbox.height = 0.03
	wheat: 82.80% bbox.left = 0.88, bbox.top = 0.00, bbox.width = 0.09, bbox.height = 0.02
	wheat: 90.95% bbox.left = 0.83, bbox.top = 0.03, bbox.width = 0.07, bbox.height = 0.06
	wheat: 57.18% bbox.left = 0.30, bbox.top = 0.15, bbox.width = 0.07, bbox.height = 0.07
	wheat: 60.73% bbox.left = 0.71, bbox.top = 0.45, bbox.width = 0.08, bbox.height = 0.09
	wheat: 91.62% bbox.left = 0.35, bbox.top = 0.51, bbox.width = 0.08, bbox.height = 0.08
	wheat: 75.67% bbox.left = 0.80, bbox.top = 0.53, bbox.width = 0.06, bbox.height = 0.08
	wheat: 62.55% bbox.left = 0.80, bbox.top = 0.60, bbox.width = 0.05, bbox.height = 0.06
	wh

PREDICTION FOR f5a1f0358.jpg
	wheat: 94.96% bbox.left = 0.00, bbox.top = 0.01, bbox.width = 0.06, bbox.height = 0.05
	wheat: 64.91% bbox.left = 0.40, bbox.top = 0.01, bbox.width = 0.06, bbox.height = 0.03
	wheat: 72.19% bbox.left = 0.50, bbox.top = 0.00, bbox.width = 0.09, bbox.height = 0.03
	wheat: 96.29% bbox.left = 0.65, bbox.top = 0.11, bbox.width = 0.08, bbox.height = 0.08
	wheat: 63.46% bbox.left = 0.00, bbox.top = 0.14, bbox.width = 0.01, bbox.height = 0.07
	wheat: 97.91% bbox.left = 0.40, bbox.top = 0.16, bbox.width = 0.07, bbox.height = 0.06
	wheat: 97.25% bbox.left = 0.15, bbox.top = 0.25, bbox.width = 0.07, bbox.height = 0.07
	wheat: 64.45% bbox.left = 0.48, bbox.top = 0.29, bbox.width = 0.06, bbox.height = 0.06
	wheat: 57.27% bbox.left = 0.21, bbox.top = 0.31, bbox.width = 0.09, bbox.height = 0.07
	wheat: 96.86% bbox.left = 0.47, bbox.top = 0.30, bbox.width = 0.06, bbox.height = 0.07
	wheat: 91.33% bbox.left = 0.54, bbox.top = 0.39, bbox.width = 0.07, bbox.height = 0.07
	wh